# Sandbox for `process`

Sandbox notebook for `process` to see what the functions do and test different parameters.

In [1]:
import os

os.chdir("../")
from scripts.analysis_tools_cython import *
import random

In [2]:
data, lc_info = import_lightcurve(
    "betapic/fits/hlsp_tess-spoc_tess_phot_0000000270577175-s0006_tess_v1_lc.fits"
)

---
### Process the lightcurve

In [3]:
flat_flux, trend_flux = smoothing(data, method="median")
a = Table()
a["time"] = data[data.colnames[0]]
a["flux"] = flat_flux - np.ones(len(flat_flux))
a["quality"] = data[data.colnames[2]]
a["flux_error"] = data[data.colnames[3]]
# data[data.colnames[1]] = flat_flux - np.ones(len(flat_flux)) # resets normalisation to zero instead of one.
t, flux, quality, real, flux_error = clean_data(a)
flux *= real

timestep = calculate_timestep(data)
factor = (1 / 48) / timestep
# now throw away interpolated points (we're reprocessing
# and trying to get the shape parameters right)
# t = t[np.array(real,dtype=bool)]
# flux = flux[np.array(real,dtype=bool)]
# quality = quality[np.array(real,dtype=bool)]
# real = real[np.array(real,dtype=bool)]
N = len(t)
ones = np.ones(N)
# flux = normalise_flux(flux)

## fourier and Lomb-Scargle
A_mag = np.abs(np.fft.rfft(normalise_flux(flux)))

freq, powers = LombScargle(t, flux).autopower()  # think about that one
peak_power = powers.max()

## M-statistic
M_stat = calc_mstatistic(flux)

T1 = test_statistic_array(flux, 60 * factor)

m, n = np.unravel_index(
    T1.argmin(), T1.shape
)  # T.argmin(): location of  T.shape: 2D array with x,y points in that dimension
minT = T1[m, n]
# Ts = nonzero(T1[m,n]).std()
minT_time = t[n]
minT_duration = m * timestep
Tm_start = n - math.floor((m - 1) / 2)
Tm_end = Tm_start + m
Tm_depth = flux[Tm_start:Tm_end].mean()
Ts = nonzero(T1[m]).std()  # only the box width selected. Not RMS of all T-statistic

In [4]:
def calc_shape_test(
    m, n, time, flux, quality, flux_error, n_m_bg_start=1, n_m_bg_scale_factor=2
):
    """Fit both symmetric and comet-like transit profiles and compare fit.
    Returns:
    (1) Asymmetry: ratio of (errors squared)
    Possible errors and return values:
    -1 : Divide by zero as comet profile is exact fit
    -2 : Too close to end of light curve to fit profile
    -3 : Unable to fit model (e.g. timeout)
    -4 : Too much empty space in overall light curve or near dip
    (2,3) Widths of comet curve fit segments.
    info: t, x, q, fit1 and fit2 are the transit shape elements

    """
    ## how many transit widths to take the general linear trend from. start is 1/4 length of cutout from beginning, end is 1 from end.
    # first_index = n - (n_m_bg_start*n)
    # last_index = n - (n_m_bg_end*m)

    ## the transit widths of the cutout from the T-statistic minimum value.
    ## this project requires the cutout to have more transit widths after the midtransit, to cover more of the tail.
    ## default is set to 1 transit width before and 2 transit widths after

    n_m_bg_end = n_m_bg_scale_factor * n_m_bg_start

    cutout_before = n - (m * n_m_bg_start)
    cutout_after = n + (m * n_m_bg_end)

    if cutout_before >= 0 and cutout_after < len(time):
        t = time[cutout_before:cutout_after]
        if (t[-1] - t[0]) / np.median(np.diff(t)) / len(t) > 1.5:
            return -4, -4, -4, 0, -4
        t0 = time[n]
        diffs = np.diff(t)

        x = flux[cutout_before:cutout_after]
        q = quality[cutout_before:cutout_after]
        fe = flux_error[cutout_before:cutout_after]

        bg_before = np.mean(x[: int(m / 4)])
        bg_time_before = np.mean(t[: int(m / 4)])
        bg_after = np.mean(x[-int(round(m / 4)) :])
        bg_time_after = np.mean(t[-int(round(m / 4)) :])

        grad = (bg_after - bg_before) / (bg_time_after - bg_time_before)
        background_level = bg_before + grad * (t - bg_time_before)
        x = x - background_level
        try:
            params1, pcov1 = single_gaussian_curve_fit(t, -x)
            params2, pcov2 = comet_curve_fit(t, -x)
        except:
            return -3, -3, -3, 0, -3

        fit1 = -gauss(t, *params1)
        fit2 = -comet_curve(t, *params2)
        depth = fit2.min()
        scores = [score_fit(x, fit) for fit in [fit1, fit2]]
        skewness = skew(-x)
        plt.plot(t, fit2)
        if scores[1] > 0:
            return (
                scores[0] / scores[1],
                params2[2],
                params2[3],
                depth,
                [t, x, q, fe, fit1, fit2, background_level, skewness],
            )
        else:

            return -1, -1, -1, 0, -1
    else:

        return -2, -2, -2, 0, -2

In [5]:
asym, width1, width2, depth, info = calc_shape_test(m, n, t, flux, quality, flux_error)

NameError: name 'skew' is not defined